<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFEpostProcessingRAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Installation
---



In [ ]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

Thu Jul 23 21:39:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Core(s) per socket:  1
Thread(s) per core:  2


In [ ]:
%%time
# %%capture
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [ ]:
%%time
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
!pip install memory_profiler
# install parquet libraries
!pip install git+https://github.com/dask/fastparquet
!pip install python-snappy
!pip install pyarrow 

In [ ]:
from pathlib import Path
import os

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box

In [ ]:
import cudf
import cugraph
import cuspatial
import dask_cudf

#Post-processing process 

1.   Load and concatenate parquet files into dask_cudf
2.   Count 'Source"
1.   Merge with Shapefile (gdf)
1.   Create output data









In [ ]:
# Paths
pathShapefile = Path('/content/drive/My Drive/Colab Notebooks/01_Repository/FFE/data/shapefile')
pathParquets = Path('/content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison')

os.listdir(pathParquets)

In [ ]:
# Load and concatenate parquet files into dask_cudf


In [ ]:
eCu = cudf.read_parquet(p / 'Copy of edge_data.parquet')
ePd = pd.read_parquet(p / 'Copy of edge_data.parquet')

In [ ]:
type(eCu), type(ePd)

(cudf.core.dataframe.DataFrame, pandas.core.frame.DataFrame)

In [ ]:
eCu.columns

Index(['source', 'target', 'distance', 'bearing', 'IgnProb_bl'], dtype='object')

In [ ]:
%time
test_eCu = eCu['IgnProb_bl'] * 100 / 4.252

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 16.9 µs


In [ ]:
%time
test_ePd = ePd['IgnProb_bl'] * 100 / 4.252

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


In [ ]:
eCuDask = dask_cudf.from_cudf(eCu, npartitions=2)

In [ ]:
%time
test_eCuDask = eCuDask['IgnProb_bl'] * 100 / 4.252

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs


In [ ]:
%time
Concat_ePd = pd.concat([ePd,ePd])

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 51.3 µs


In [ ]:
%time
Concat_eCu = cudf.concat([eCu,eCu])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [ ]:
%time
Concat_eCuDask = dask_cudf.concat([eCuDask, eCuDask]).compute()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [ ]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

Thu Jul  9 21:00:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    43W / 250W |   1789MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------